## Gerenciamento de Pacotes de Equipamentos

Este projeto Python foi desenvolvido para auxiliar no gerenciamento e controle de pacotes de equipamentos dos FPSOs P84 e P85.


### Importa modulos

In [106]:
import pandas as pd
from pathlib import Path
import re
from datetime import datetime

#### Ignorando as mensagens de aviso ("Deprecation Warning") no python

In [107]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### Dados Básicos

In [108]:
data_dados_basicos = {
'PKG_DESCRIPTION': ['API 610 CENTRIFUGAL PUMPS', 'DECK TROLLEY', 'FLARE SYSTEM', 'FRESH WATER CHLORINATION UNIT', 'NITROGEN GENERATOR UNITS', 'NON-API 610 CENTRIFUGAL PUMPS', 'PIG LAUNCHERS AND RECEIVERS', 'PROGRESSIVE CAVITY PUMPS', 'RECIPROCATING PUMPS', 'SEA WATER ELECTROCHLORINATION UNIT', 'SEA WATER LIFT PUMP', 'SHELL AND TUBE HEAT EXCHANGERS (Himile)', 'SHELL AND TUBE HEAT EXCHANGERS (Asvotec)', 'WATER INJECTION PUMPS', 'FRESH WATER MAKER FOR OIL DILUTION', 'PRINTED CIRCUIT HEAT EXCHANGER', 'OFFSHORE CRANE'],
'Discipline': ['ROE', 'ROE', 'ROE', 'STATIC', 'ROE', 'ROE', 'STATIC', 'ROE', 'ROE', 'STATIC', 'ROE', 'STATIC', 'STATIC', 'ROE', 'STATIC', 'STATIC', 'ROE'],
'Critical': ['', '', 'Critical', '', '', '', '', '', 'Critical', '', 'Super Critical', '', '', 'Super Critical', '', 'Critical', 'Critical'],
'System': ['1200,1223,5124,5125,5331,5115', '5266', '5412', '5122', '5241', '120N,1200,5115,5124,5125', '1244,1231,1210', '5412', '5115,5133', '5121', '5111', '1223', '1200,1223,1251,1350,5125,5135,5147,1225,1231', '1251', '5122', '1231,1252,1254', '5266'],
'Trigram': ['KD9', 'x', 'x', 'D5A', 'GK1', 'KD9', 'AP5', 'NZA', 'KFQ', 'D5A', 'FM2', 'HM9', 'ATI', 'SJA', 'HE1', 'HLT', 'ND1'],
'Vendor': ['KSB BRASIL LTDA', '', '', 'DE NORA DO BRASIL LTDA', 'GARDNER DENVER KOREA', 'KSB BRASIL LTDA', 'APPLIED ENGINEERING PTE LTD', 'Netzsch Asia Pacific Pte Ltd', 'PERONI POMPE SPA', 'DE NORA DO BRASIL LTDA', 'FRAMO AS', 'HIMILE MECHANICAL MANUFACTURING (SHANDONG) CO., LTD', 'ASVOTEC TERMOINDUSTRIAL LTDA', 'SULZER BRASIL SA', 'HITACHI AQUA TECH (HAQT)E1', 'HEATRIC DIVISION OF MEGGITT (UK) LIMITED', 'NATIONAL OILWELL VARCO NORWAY AS'],
'MR_Number': ['I-RM-3010.2S-1200-311-S2N-002', 'I-RM-3010.2S-5266-620-S2N-001', 'I-RM-3010.2S-5412-583-S2N-001', 'I-RM-3010.2S-5122-560-S2N-042', 'I-RM-3010.2S-5241-470-S2N-001', 'I-RM-3010.2S-1200-311-S2N-001', 'I-RM-3010.2S-1210-296-S2N-001', 'I-RM-3010.2S-1200-313-S2N-001', 'I-RM-3010.2S-5133-312-S2N-001', 'I-RM-3010.2S-5121-560-S2N-040', 'I-RM-3010.2S-5111-311-S2N-001', 'I-RM-3010.2S-1200-451-S2N-011', 'I-RM-3010.2S-1200-451-S2N-011', 'I-RM-3010.2S-1251-311-S2N-001', 'I-RM-3010.2S-5122-580-S2N-040', 'I-RM-3010.2S-1200-459-S2N-032', 'I-RM-3010.2S-5266-631-S2N-001'],
'TBE_Number': ['I-PT-3010.2S-1200-311-S2N-002', 'I-PT-3010.2S-5266-620-S2N-001', 'I-PT-3010.2S-5412-583-S2N-001', 'I-PT-3010.2S-5122-560-S2N-042', 'I-PT-3010.2S-5241-470-S2N-001, I-PT-3010.2S-5241-470-S2N-002', 'I-PT-3010.2S-1200-311-S2N-001', 'I-PT-3010.2S-1210-296-S2N-018', 'I-PT-3010.2S-1200-313-S2N-001', 'I-PT-3010.2S-5133-312-S2N-001', 'I-PT-3010.2S-5121-560-S2N-040', 'I-PT-3010.2S-5111-311-S2N-001', 'I-PT-3010.2S-1200-451-S2N-011', 'I-PT-3010.2S-1200-451-S2N-011', 'I-PT-3010.2S-1251-311-S2N-001', 'I-PT-3010.2S-5122-580-S2N-040', 'I-PT-3010.2S-1200-459-S2N-032', 'I-PT-3010.2S-5266-631-S2N-001'],
'KOM_DATE': ['', '', '', '10/03/2025', '18/02/2025', '', '24/03/2025', '', '', '07/03/2025', '17-18/02/2025', '21/03/2025', '19/03/2025', '21-22/10/2025', '25/03/2025', '','24/03/2025'],
'PIM_DATE': ['', '', '', '', '01-02/04/2025', '', 's 08/2025', '', '', '', '', '', '', '17/01/2025', '', '', ''],
'DR_MOTORS_DATE': ['', '', '', '', '', '', '', '', '', '', '', '', '', '17/03/2025', '', '',''],
'PLAN_DELIVERY_DATE': ['', '', '', '28/02/2026', '30/10/2025 & 28/02/2026', '', '', '', '', '28/02/2026', '01/01/2027', '15/03/2026', '15/03/2026', '03/04/2026', '', '',''],
}

df_db = pd.DataFrame(data_dados_basicos)
print('\nDADOS BÁSICOS DOS PACOTES:')
display(df_db)


DADOS BÁSICOS DOS PACOTES:


,PKG_DESCRIPTION,Discipline,Critical,System,Trigram,Vendor,MR_Number,TBE_Number,KOM_DATE,PIM_DATE,DR_MOTORS_DATE,PLAN_DELIVERY_DATE
0,API 610 CENTRIFUGAL PUMPS,ROE,,"1200,1223,5124,5125,5331,5115",KD9,KSB BRASIL LTDA,I-RM-3010.2S-1200-311-S2N-002,I-PT-3010.2S-1200-311-S2N-002,,,,
1,DECK TROLLEY,ROE,,5266,x,,I-RM-3010.2S-5266-620-S2N-001,I-PT-3010.2S-5266-620-S2N-001,,,,
2,FLARE SYSTEM,ROE,Critical,5412,x,,I-RM-3010.2S-5412-583-S2N-001,I-PT-3010.2S-5412-583-S2N-001,,,,
3,FRESH WATER CHLORINATION UNIT,STATIC,,5122,D5A,DE NORA DO BRASIL LTDA,I-RM-3010.2S-5122-560-S2N-042,I-PT-3010.2S-5122-560-S2N-042,10/03/2025,,,28/02/2026
4,NITROGEN GENERATOR UNITS,ROE,,5241,GK1,GARDNER DENVER KOREA,I-RM-3010.2S-5241-470-S2N-001,"I-PT-3010.2S-5241-470-S2N-001, I-PT-3010.2S-52...",18/02/2025,01-02/04/2025,,30/10/2025 & 28/02/2026
5,NON-API 610 CENTRIFUGAL PUMPS,ROE,,"120N,1200,5115,5124,5125",KD9,KSB BRASIL LTDA,I-RM-3010.2S-1200-311-S2N-001,I-PT-3010.2S-1200-311-S2N-001,,,,
6,PIG LAUNCHERS AND RECEIVERS,STATIC,,"1244,1231,1210",AP5,APPLIED ENGINEERING PTE LTD,I-RM-3010.2S-1210-296-S2N-001,I-PT-3010.2S-1210-296-S2N-018,24/03/2025,s 08/2025,,
7,PROGRESSIVE CAVITY PUMPS,ROE,,5412,NZA,Netzsch Asia Pacific Pte Ltd,I-RM-3010.2S-1200-313-S2N-001,I-PT-3010.2S-1200-313-S2N-001,,,,
8,RECIPROCATING PUMPS,ROE,Critical,"5115,5133",KFQ,PERONI POMPE SPA,I-RM-3010.2S-5133-312-S2N-001,I-PT-3010.2S-5133-312-S2N-001,,,,
9,SEA WATER ELECTROCHLORINATION UNIT,STATIC,,5121,D5A,DE NORA DO BRASIL LTDA,I-RM-3010.2S-5121-560-S2N-040,I-PT-3010.2S-5121-560-S2N-040,07/03/2025,,,28/02/2026


### Função para extrair sistemas e trigramas do dataframe df_db

In [109]:
import pandas as pd
import re

# Função para extrair sistemas e trigramas do dataframe df_db

def extrair_sistemas_trigramas(df):
    # Inicializar um dicionário para armazenar relações sistema-trigrama
    relacoes = {}
    
    # Iterar sobre cada linha do dataframe
    for _, row in df.iterrows():
        # Obter o trigrama para esta linha
        trigram = row['Trigram']
        
        # Limpar e dividir a string de sistemas
        systems_str = row['System']
        # Substituir espaços antes ou depois de vírgulas
        systems_str = re.sub(r'\s*,\s*', ',', systems_str)
        # Dividir a string pelos separadores de vírgula
        systems = systems_str.split(',')
        
        # Para cada sistema nesta linha, associar ao trigrama
        for system in systems:
            system = system.strip()
            if system:  # Verificar se o sistema não está vazio
                if system not in relacoes:
                    relacoes[system] = set()
                relacoes[system].add(trigram)
    
    # Converter o dicionário para o formato de lista de tuplas
    sistemas_trigramas = []
    for system, trigrams in relacoes.items():
        for trigram in trigrams:
            sistemas_trigramas.append((system, trigram))
    
    # Ordenar a lista para melhor legibilidade
    sistemas_trigramas.sort()
    
    return sistemas_trigramas


# Extrair os sistemas e trigramas
sistemas_trigramas = extrair_sistemas_trigramas(df_db)

# Imprimir o resultado formatado como no exemplo
print("# Dicionário com os sistemas e trigramas")
print("sistemas_trigramas = [")
for sistema, trigrama in sistemas_trigramas:
    print(f"    ('{sistema}', '{trigrama}'),")
print("    # Adicione mais sistemas e trigramas conforme necessário")
print("]")


# Dicionário com os sistemas e trigramas
sistemas_trigramas = [
    ('1200', 'ATI'),
    ('1200', 'KD9'),
    ('120N', 'KD9'),
    ('1210', 'AP5'),
    ('1223', 'ATI'),
    ('1223', 'HM9'),
    ('1223', 'KD9'),
    ('1225', 'ATI'),
    ('1231', 'AP5'),
    ('1231', 'ATI'),
    ('1231', 'HLT'),
    ('1244', 'AP5'),
    ('1251', 'ATI'),
    ('1251', 'SJA'),
    ('1252', 'HLT'),
    ('1254', 'HLT'),
    ('1350', 'ATI'),
    ('5111', 'FM2'),
    ('5115', 'KD9'),
    ('5115', 'KFQ'),
    ('5121', 'D5A'),
    ('5122', 'D5A'),
    ('5122', 'HE1'),
    ('5124', 'KD9'),
    ('5125', 'ATI'),
    ('5125', 'KD9'),
    ('5133', 'KFQ'),
    ('5135', 'ATI'),
    ('5147', 'ATI'),
    ('5241', 'GK1'),
    ('5266', 'ND1'),
    ('5266', 'x'),
    ('5331', 'KD9'),
    ('5412', 'NZA'),
    ('5412', 'x'),
    # Adicione mais sistemas e trigramas conforme necessário
]


#### Scope of Supply

In [110]:
# Definição do dataframe do pacote para WATER INJECTION PUMPS AND SEA WATER BOOSTER PUMPS

data_wip = {
'TAG': ['B-1251001A/C', 'M-B-1251001A/C', 'B-1251002A/C', 'M-B-1251002A/C', 'B-1251003A/B', 'M-B-1251003A/B', 'VV-B-1251002A/C', 'PN-B-1251002A/C'],
'QTY': ['3 x 50%', '3 set', '3 x 50%', '3 set', '2 x 100%', '2 set', '3 set', '3 set'],
'DESCRIPTION': ['INJECTION WATER BOOSTER PUMP', 'INJECTION WATER BOOSTER PUMP - MOTOR', 'INJECTION WATER MAIN PUMP', 'INJECTION WATER MAIN PUMP - MOTOR', 'SEAWATER BOOSTER PUMP', 'SEAWATER BOOSTER PUMP - MOTOR', 'HVSD FOR MAIN INJECTION WATER PUMP', 'MAIN INJECTION WATER PUMP CONTROL PANEL'],
'LOCATION': ['M-11', 'M-11', 'M-11', 'M-11', 'M-14', 'M-14', 'M-11', 'M-11'],
'AREA_CLASSIFICATION': ['Outdoor, Safe Area', 'Outdoor, Safe Area', 'Outdoor, Safe Area', 'Outdoor, Safe Area', 'Outdoor, Zone 2, T3, Group IIA', 'Outdoor, Zone 2, T3, Group IIA', 'Indoor, Safe Area', 'Indoor, Safe Area'],
'PKG_DESCRIPTION': ['WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS']
}

df_wip = pd.DataFrame(data_wip)
print('Scope of Supply for WATER INJECTION PUMPS AND SEA WATER BOOSTER PUMPS')
# print(df_wip)

# Definição do dataframe do pacote para WATER LIFT PUMPS (B-5111001A/D AND B-5111002)
data_lift_pump = {
'TAG': ['B-5111001A/D', 'M-B-5111001A/D', 'B-5111002', 'M-B-5111002', 'PN-B-5111001A/D', 'PN-B-5111002-01', 'PN-B-5111002-02', 'UB-B-5111001A/D', 'UB-B-5111002'],
'QTY': ['4 x 33%', '4 x 33%', '1 x 100%', '1 x 100%', '4', '1', '1', '4', '1'],
'DESCRIPTION': ['SEA WATER LIFT PUMP', 'ELECTRIC MOTOR FOR SEA WATER LIFT PUMP', 'START-UP SEA WATER LIFT PUMP', 'ELECTRIC MOTOR FOR START-UP SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP CONTROL PANEL', 'START-UP SEA WATER LIFT PUMP CONTROL PANEL', 'START-UP SEA WATER LIFT PUMP POWER PANEL', 'FLUID CIRCULATION UNIT FOR SEA WATER LIFT PUMPS', 'FLUID CIRCULATION UNIT FOR START-UP SEA WATER LIFT PUMP'],
'LOCATION': ['MAIN DECK', 'MAIN DECK', 'MAIN DECK', 'MAIN DECK', 'M-17', 'M-17', 'ENGINE ROOM', 'MAIN DECK', 'MAIN DECK'],
'AREA_CLASSIFICATION': ['SAFE AREA', 'SAFE AREA', 'SAFE AREA', 'SAFE AREA', 'SAFE AREA', 'SAFE AREA', 'SAFE AREA', 'ZONE 1, IIA, T3', 'ZONE 1, IIA, T3'],
'PKG_DESCRIPTION': ['SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP']
}

df_lift_pump = pd.DataFrame(data_lift_pump)
print('Scope of Supply for WATER LIFT PUMPS (B-5111001A/D AND B-5111002)')
# print(df_lift_pump)

# Definição do dataframe do pacote para WATER ELECTROCHLORINATION UNIT

data_electrochlorination = {
'TAG': ['UE-5121501', 'PN-UE-5121501'],
'QTY': ['1 x 100%', '1 x 100%'],
'DESCRIPTION': ['Sea Water Electrochlorination Unit', 'Sea Water Electrochlorination Unit Control Panel'],
'LOCATION': ['M-15', 'M-15'],
'AREA_CLASSIFICATION': ['Outdoor, Safe Area', 'Outdoor, Safe Area'],
'PKG_DESCRIPTION': ['SEA WATER ELECTROCHLORINATION UNIT', 'SEA WATER ELECTROCHLORINATION UNIT']
}

df_electrochlorination = pd.DataFrame(data_electrochlorination)
print('Scope of Supply for SEA WATER ELECTROCHLORINATION UNIT')
# print(df_electrochlorination)

# Definição do dataframe do pacote para NITROGEN GENERATOR UNITS

data_N2_generation = {
'TAG': ['Z-5241001A/B', 'PN-Z-5241001A/B', 'TBA', 'TBA', 'V-Z-5241001', 'TBA', 'TBA', 'Z-5241002A/B', 'PN-Z-5241002A/B', 'TBA', 'TBA', 'TBA', 'TBA'],
'QTY': ['2 x 100%', '2 x 100%', '2 x 100%', '2 x 100%', '1 x 100%', '1 x 100%', '130 bottles', '2 x 100%', '2 x 100%', '2 x 100%', '2 x 100%', '2 x 100%', '2 x 100%'],
'DESCRIPTION': ['Nitrogen Generator Unit (Filters and membrane modules)', 'Nitrogen Generator Unit Control Panel', 'Nitrogen Compressor', 'Vent Including Silencer', 'Nitrogen Buffer Vessel', 'Nitrogen Intensifier Unit', 'Emergency Bottles Battery (50L) in skidded units', 'Nitrogen Generator Unit for Flare (Filters and membrane modules)', 'Nitrogen Generator Unit for Flare Control Panel', 'Air Compressors', 'Air Dryers with filters', 'Vent Including Silencer', 'Oxygen Content Analyzer'],
'LOCATION': ['M-01', 'M-01', 'M-01', 'M-01', 'M-01', 'M-01', 'M-01', 'Forecastle (N2 Gen. Room)', 'Forecastle (N2 Gen. Room)', 'Forecastle (N2 Gen. Room)', 'Forecastle (N2 Gen. Room)', 'Forecastle (N2 Gen. Room)', 'Control Room'],
'AREA_CLASSIFICATION': ['Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Safe', 'Safe', 'Safe', 'Safe', 'Safe', 'Safe'],
'PKG_DESCRIPTION': ['NITROGEN GENERATOR UNITS', 'NITROGEN GENERATOR UNITS', 'NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS']
}

df_N2_generation = pd.DataFrame(data_N2_generation)
print('Scope of Supply for NITROGEN GENERATOR UNITS')
# print(df_N2_generation)

# Definição do dataframe do pacote para SHELL AND TUBE HEAT EXCHANGERS
data_heat_exchangers = {
'TAG': ['P-1223001A/F', 'P-1223003A/B', 'P-1223004', 'P-1251001A/B', 'P-1350001A/B', 'P-5125001', 'P-5135001', 'P-5135002', 'P-5147001', 'P-UC-1225001A/B-01', 'P-UC-1225001A/B-02', 'P-UC-1231001A/C-01', 'P-UC-1231001A/C-02'],
'QTY': ['6 x 25%', '2 x 50%', '1 x 100%', '2 x 50%', '2 x 100%', '1 x 100%', '1 x 100%', '1 x 100%', '1 x 100%', '2 x 100%', '2 x 100%', '3 x 50%', '3 x 50%'],
'DESCRIPTION': ['OIL/OIL PRE HEATER', 'DILUTION WATER HEATER', 'TEST HEATER', 'INJECTION WATER RECYCLE COOLER', 'STRUCTURAL TANKS RECOVERY GAS COOLER', 'UTILITY HOT WATER HEATER', 'FUEL GAS HEATER', 'FUEL GAS SUPER HEATER', 'HOT WATER DUMP COOLER', 'VRU 1ST STAGE SUCTION COOLER', 'VRU 1ST STAGE DISCHARGE COOLER', 'MAIN GAS COMPRESSOR SUCTION COOLER', 'MAIN GAS COMPRESSOR DISCHARGE COOLER'],
'LOCATION': ['M-10A', 'M-10C', 'M-10B', 'M-11', 'M-01', 'M-15', 'M-06', 'M-06', 'M-15', 'M-05B', 'M-05B', 'M-05', 'M-05'],
'AREA_CLASSIFICATION': ['M-10C', 'UNCLASSIFIED', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'UNCLASSIFIED', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'UNCLASSIFIED', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'UNCLASSIFIED', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3'],
'PKG_DESCRIPTION': ['SHELL AND TUBE HEAT EXCHANGERS (Asvotec)'] * 13
}

df_heat_exchangers = pd.DataFrame(data_heat_exchangers)
print('Scope of Supply for SHELL AND TUBE HEAT EXCHANGERS (Asvotec)')
# display(df_heat_exchangers)

# Definição do dataframe do pacote para SHELL AND TUBE HEAT EXCHANGERS - Production
data_heat_exchangers_prod = {
'TAG': ['P-1223002A/F'],
'QTY': ['6 x 25%'],
'DESCRIPTION': ['PRODUCTION HEATER'],
'LOCATION': ['M-10B'],
'AREA_CLASSIFICATION': ['ZONE 2; GAS GROUP IIA, TEMP CLASS T3'],
'PKG_DESCRIPTION': ['SHELL AND TUBE HEAT EXCHANGERS (Himile)']
}

df_heat_exchangers_prod = pd.DataFrame(data_heat_exchangers_prod)
print('Scope of Supply for SHELL AND TUBE HEAT EXCHANGERS (Himile)')
# display(df_heat_exchangers_prod)

# Definição do dataframe do pacote para PROGRESSIVE CAVITY PUMPS

data_progressive_pump = {
'TAG': ['B-5412001A/B', 'M-B-5412001A/B', 'B-5412002A/B', 'M-B-5412002A/B'],
'QTY': ['2 X 100%', '2 X 100%', '2 X 100%', '2 X 100%'],
'DESCRIPTION': ['HIGH PRESSURE FLARE K. O. DRUM PUMP', 'ELECTRIC MOTOR FOR HIGH PRESSURE FLARE K. O. DRUM PUMP', 'LOW PRESSURE FLARE K. O. DRUM PUMP', 'ELECTRIC MOTOR FOR LOW PRESSURE FLARE K. O. DRUM PUMP'],
'LOCATION': ['M-01 PRODUCTION DECK -LEVEL 1', 'M-01 PRODUCTION DECK -LEVEL 1', 'M-01 PRODUCTION DECK -LEVEL 1', 'M-01 PRODUCTION DECK -LEVEL 1'],
'AREA_CLASSIFICATION': ['ZONE 2, GROUP llA, T3', 'ZONE 2, GROUP llA, T3', 'ZONE 2, GROUP llA, T3', 'ZONE 2, GROUP llA, T3'],
'PKG_DESCRIPTION': ['PROGRESSIVE CAVITY PUMPS','PROGRESSIVE CAVITY PUMPS','PROGRESSIVE CAVITY PUMPS','PROGRESSIVE CAVITY PUMPS']
}

df_progressive_pump = pd.DataFrame(data_progressive_pump)
print('Scope of Supply for PROGRESSIVE CAVITY PUMPS')
# print(df_progressive_pump)

# Definição do dataframe do pacote para PIG LAUNCHERS AND RECEIVERS

data_PIG = {
'TAG': ['LP-1244001A/J', 'LP-1244002A/E', 'LP-1244003A/G', 'LR-1231001', 'RP-1210001A/B', 'RP-1210002', 'RP-1210003A/B'],
'QTY': ['9', '5', '7', '1', '2', '1', '2'],
'DESCRIPTION': ['Satellite Production Well Pig Launcher', 'PWAGFLEX Well Pig Launcher', 'WAG-LOOP Well Pig Launcher', 'Gas Pipeline Pig Launcher/Receiver', 'Satellite Production Well Pig Receiver', 'PWAGFLEX Well Pig Receiver', 'WAG-LOOP Well Pig Receiver'],
'LOCATION': ['M-09', 'M-09', 'M-09', 'M-09', 'M-09', 'M-09', 'M-09'],
'AREA_CLASSIFICATION': ['Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3'],
'PKG_DESCRIPTION': ['PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS']
}

df_PIG = pd.DataFrame(data_PIG)
print('Scope of Supply for PIG LAUNCHERS AND RECEIVERS')
# print(df_PIG)

# Definição do dataframe do pacote para NON-API 610 CENTRIFUGAL PUMPS

data_non_api_pump = {
'TAG': ['B-5115004A/B', 'M-B-5115004A/B', 'B-5124002A/C', 'M-B-5124002A/C', 'B-5124003', 'M-B-5124003', 'PN-B-5124003', 'B-5125002 A/B', 'M-B-5125002 A/B'],
'QTY': ['2', '2', '3', '3', '1', '1', '1', '2', '2'],
'DESCRIPTION': ['FLARE AND SLOP VESSEL FRESH WATER MAKE-UP PUMP', 'ELECTRIC MOTOR FOR FLARE AND SLOP VESSEL FRESH WATER MAKE-UP PUMP', 'COOLING WATER CIRCULATION PUMP - NON-CLASSIFIED AREA', 'ELECTRIC MOTOR FOR COOLING WATER CIRCULATION PUMP - NON-CLASSIFIED AREA', 'START-UP COOLING WATER PUMP - NON-CLASSIFIED AREA', 'ELECTRIC MOTOR FOR START-UP COOLING WATER PUMP - NON-CLASSIFIED AREA', 'START-UP COOLING WATER PUMP - NON-CLASSIFIED AREA POWER PANEL', 'UTILITY HOT WATER CIRCULATION PUMP', 'ELECTRICAL MOTOR FOR UTILITY HOT WATER CIRCULATION PUMP'],
'LOCATION': ['M-15', 'M-15', 'M-14', 'M-14', 'M-14', 'M-14', 'ENGINE ROOM', 'M-15', 'M-15'],
'AREA_CLASSIFICATION': ['UNCLASSIFIED'] * 2 + ['ZONE 2, GROUP IIA, T3 (for running during ESD-3P)'] * 2 + ['UNCLASSIFIED'] * 5,
'PKG_DESCRIPTION': ['NON-API 610 CENTRIFUGAL PUMPS'] * 9
}

df_non_api_pump = pd.DataFrame(data_non_api_pump)
print('Scope of Supply for NON-API 610 CENTRIFUGAL PUMPS')
# print(df_non_api_pump)

# Definição do dataframe do pacote para API 610 CENTRIFUGAL PUMPS
data_api_pump = {
'TAG': ['B-1223001 A/B', 'B-1223001 C/D', 'M-B-1223001 A/B', 'M-B-1223001 C/D', 'CF-B-1223001 A/B/C/D', 'B-1223002A/B', 'B-1223002C/D', 'M-B-1223002A/B', 'M-B-1223002C/D', 'B-1223003A/B', 'B-1223003C/D', 'M-B-1223003A/B', 'M-B-1223003C/D', 'CF-B-1223003A/B/C/D', 'B-1223004A/B', 'M-B-1223004A/B', 'B-5124001A/B/C/D', 'M-B-5124001A/B/C/D', 'B-5125001A/B/C', 'M-B-5125001A/B/C', 'B-5125003A/B', 'M-B-5125003A/B', 'B-5331001A/B/C', 'M-B-5331001A/B/C', 'B-5115003A/C (HOLD  - 1)', 'M-B-5115003A/C (HOLD  - 1)'],
'QTY': ['4 X 50%', '4 X 50%', '4', '4', '4', '4 X 50%', '4 X 50%', '4', '4', '4 x 50%', '4 x 50%', '4', '4', '4', '2 X 100%', '2', '4 X 33%', '4', '3 X 50%', '3', '2 X 100%', '2', '3 X 50%', '3', '3 x 50%', '3'],
'DESCRIPTION': ['PRE-OIL DEHYDRATOR  RECIRCULATION WATER  PUMP (OH2 TYPE)', 'PRE-OIL DEHYDRATOR  RECIRCULATION WATER  PUMP (OH2 TYPE)', 'ELECTRIC MOTOR FOR  PRE-OIL DEHYDRATOR  RECIRCULATION WATER  PUMP', 'ELECTRIC MOTOR FOR  PRE-OIL DEHYDRATOR  RECIRCULATION WATER  PUMP', 'FREQUENCY CONVERTER  FOR PRE -OIL  DEHYDRATOR  RECIRCULATION WATER  PUMP', 'OIL DEHYDRATOR  RECIRCULATION WATER  PUMP  (OH2 TYPE)', 'OIL DEHYDRATOR  RECIRCULATION WATER  PUMP  (OH2 TYPE)', 'ELECTRIC MOTOR FOR  OIL DEHYDRATOR  RECIRCULATION WATER  PUMP', 'ELECTRIC MOTOR FOR  OIL DEHYDRATOR  RECIRCULATION WATER  PUMP', 'OIL TRANSFER PUMP  (OH2 TYPE)', 'OIL TRANSFER PUMP  (OH2 TYPE)', 'ELECTRIC MOTOR FOR  OIL TRANSFER PUMP', 'ELECTRIC MOTOR FOR  OIL TRANSFER PUMP', 'FREQUENCY CONVERTER  FOR  OIL TRANSFER   PUMP', 'TEST SEPARATOR PUMP  (OH2 TYPE)', 'ELECTRIC MOTOR FOR  TEST SEPARATOR PUMP', 'COOLING WATER  CIRCULATION PUMP (BB2  TYPE) - CLASSIFIED AREA', 'ELECTRIC MOTOR FOR  COOLING WATER  CIRCULATION PUMP -  CLASSIFIED AREA', 'HOT WATER  CIRCULATION PUMP (OH2  TYPE)', 'ELECTRIC MOTOR FOR  HOT WATER  CIRCULATION PUMP', 'HOT WATER DUMP  COOLER PUMP (OH2  TYPE)', 'ELECTRIC MOTOR FOR  HOT WATER DUMP  COOLER PUMP (OH2  TYPE)', 'PRODUCED WATER  BOOSTER PUMPS (OH2  TYPE)', 'ELECTRIC MOTOR FOR  PRODUCED WATER  BOOSTER PUMPS', 'DILUTION WATER PUMP', 'ELECTRIC MOTOR FOR  DILUTION WATER PUMPS'],
'LOCATION': ['M-10A', 'M-10C', 'M-10A', 'M-10C', 'M-17', 'M-10A', 'M-10C', 'M-10A', 'M-10C', 'M-10A', 'M-10C', 'M-10A', 'M-10C', 'M-17', 'M-10B', 'M-10B', 'M-14', 'M-14', 'M-15', 'M-15', 'M-15', 'M-15', 'M-10B', 'M-10B', 'M-15', 'M-15'],
'AREA_CLASSIFICATION': ['UNCLASSIFIED SAFE AREA'] * 9 + ['ZONE 2 , GROUP  IIA , T3'] * 4 + ['UNCLASSIFIED SAFE AREA'] * 1 + ['ZONE 2 , GROUP  IIA , T3'] * 2 + ['UNCLASSIFIED SAFE AREA'] * 4 + ['ZONE 2 , GROUP IIA , T3 (for running during ESD-3P)'] * 2 + ['ZONE 2 , GROUP  IIA , T3'] * 2 + ['UNCLASSIFIED SAFE AREA'] * 2,
'PKG_DESCRIPTION': ['API 610 CENTRIFUGAL PUMPS'] * 26
}

df_api_pump = pd.DataFrame(data_api_pump)
print('Scope of Supply for API 610 CENTRIFUGAL PUMPS')
# print(df_api_pump)

# Definição do dataframe do pacote para FRESH WATER MAKER FOR OIL DILUTION
data_fresh_water_maker = {
    'TAG': ['UD-5122002A', 'PN-UD-5122002A-01', 'PN-UD-5122002A-02', 'UD-5122002B', 'PN-UD-5122002B-01', 'PN-UD-5122002B-02'],
    
    'QTY': ['1', '1', '1', '1', '1', '1'],
    
    'DESCRIPTION': ['FRESH WATER MAKER FOR OIL DILUTION - TRAIN A', 
                    'FRESH WATER MAKER FOR OIL DILUTION UNIT CONTROL PANEL', 
                    'FRESH WATER MAKER FOR OIL DILUTION POWER PANEL', 
                    'FRESH WATER MAKER FOR OIL DILUTION - TRAIN B', 
                    'FRESH WATER MAKER FOR OIL DILUTION UNIT CONTROL PANEL', 
                    'FRESH WATER MAKER FOR OIL DILUTION POWER PANEL'],
    
    'LOCATION': ['M-15', 'M-17', 'M-17', 'M-15', 'M-17', 'M-17'],
    
    'AREA_CLASSIFICATION': ['SAFE', 'SAFE', 'SAFE', 'SAFE', 'SAFE', 'SAFE'],
    
    'PKG_DESCRIPTION': ['FRESH WATER MAKER FOR OIL DILUTION', 
                        'FRESH WATER MAKER FOR OIL DILUTION', 
                        'FRESH WATER MAKER FOR OIL DILUTION', 
                        'FRESH WATER MAKER FOR OIL DILUTION', 
                        'FRESH WATER MAKER FOR OIL DILUTION', 
                        'FRESH WATER MAKER FOR OIL DILUTION']
}

df_fresh_water_maker = pd.DataFrame(data_fresh_water_maker)
print('Scope of Supply for FRESH WATER MAKER FOR OIL DILUTION')
# display(df_fresh_water_maker)]

# Definição do dataframe do pacote para DECK TROLLEY
data_deck_trolley = {
    'TAG': ['TN-5266001', 'TN-5266002A', 'TN-5266002B/C', 'TN-5266003', 'TN-5266004', 'CB-TN-5266001', 'CB-TN-5266002A/B/C'],
    'QTY': ['1', '1', '2', '1', '1', '1', '3'],
    'DESCRIPTION': ['40t Deck Trolley', '10t Deck Trolley for 10ft ISO Food Container with 40t trolley pulling capacity', '10t Deck Trolley for 10ft ISO Food Container', '10t Deck Trolley with 40t trolley pulling capacity', '5t Deck Trolley with 10t trolley pulling capacity', 'Charging Station for 40t self-propelled decktrolley & 10t self-propelled decktrolley', 'Fixed charging station'],
    'LOCATION': ['Topside', 'Module M16, M15B', 'Module M16, M15B', 'Topside', 'Topside', 'Topside', 'Module M15B'],
    'AREA_CLASSIFICATION': ['Zone 1, T3, IIA', 'Zone 2, T3, IIA', 'Zone 2, T3, IIA', 'Zone 1, T3, IIA', 'Zone 1, T3, IIA', 'Zone 1, T3, IIA', 'Non-Hazardous Area (Optional - Zone 2, T3, IIA)'],
    'PKG_DESCRIPTION': ['DECK TROLLEY'] * 7
}

df_deck_trolley = pd.DataFrame(data_deck_trolley)
print('Scope of Supply for DECK TROLLEY')
# print(df_deck_trolley)


# Definição do dataframe do pacote para FLARE SYSTEM
data_flare_system = {
    'TAG': ['TA-5412001', 'PN-TA-5412001-01', 'PN-TA-5412001-02', 'PN-TA-5412001-05', 'CN-5412001', 'Z-TA-5412001-01', 'Z-TA-5412001-02', 'PN-5412001'],
    'QTY': ['1 X 100%', '1 X 100%', '1 X 100%', '1 X 100%', '1 X 100%', '1 X 100%', '1 X 100%', '1 X 100%'],
    'DESCRIPTION': ['FLARE', 'FLARE IGNITION AND MONITORING PANEL', 'FLARE TURNDOWN CONTROL SYSTEM PANEL', 'SPARKING PELLET IGNITION SYSTEM', 'FLARE PILOT LPG CYLINDERS', 'HP FLARE STAGING SKID', 'LP FLARE STAGING SKID', 'FLARE AND SLOP VESSEL GAS RECOVERY SYSTEM RELIEF PANEL (SIL-3)'],
    'LOCATION': ['FLARE TOWER', 'M-01', 'M-17', 'M-01', 'M-01', 'M-07A Pipe Rack / M-01 (TBC)', 'M-07A Pipe Rack / M-01 (TBC)', 'M-17'],
    'AREA_CLASSIFICATION': ['', '', '', '', '', '', '', ''],
    'PKG_DESCRIPTION': ['FLARE SYSTEM'] * 8
}

df_flare_system = pd.DataFrame(data_flare_system)
print('Scope of Supply for FLARE SYSTEM')
# display(df_flare_system)

# Definição do dataframe do pacote para FRESH WATER CHLORINATION UNIT
data_fresh_water_chlorination = {
    'TAG': ['Z-5122001', 'PN-Z-5122001'],
    'QTY': ['1 x 100%', '1 x 100%'],
    'DESCRIPTION': ['Freshwater chlorination unit', 'Freshwater chlorination unit control panel'],
    'LOCATION': ['M-15', 'M-15'],
    'AREA_CLASSIFICATION': ['SAFE', 'SAFE'],
    'PKG_DESCRIPTION': ['FRESH WATER CHLORINATION UNIT'] * 2
}

df_fresh_water_chlorination = pd.DataFrame(data_fresh_water_chlorination)
print('Scope of Supply for FRESH WATER CHLORINATION UNIT')
# display(df_fresh_water_chlorination)

# Definição do dataframe do pacote para RECIPROCATING PUMPS
data_reciprocating_pumps = {
    'TAG': ['B-5115001A/B', 'M-B-5115001A/B', 'B-5115002A/B', 'M-B-5115002A/B', 
            'B-5133001A/C', 'M-B-5133001A/C', 'PN-B-5133001A/C', 'CF-B-5133001A/C',
            'B-5133002A/B', 'M-B-5133002A/B', 'PN-B-5133002A/B', 
            'B-5133003', 'M-B-5133003'],
    'QTY': ['2 (1W+1S)', '2 (1W+1S)', '2 (1W+1S)', '2 (1W+1S)', 
            '3 (2W+1S)', '3 (2W+1S)', '3 (2W+1S)', '3 (2W+1S)',
            '2 (1W+1S)', '2 (1W+1S)', '2 (1W+1S)', 
            '1', '1'],
    'DESCRIPTION': ['HOT WATER MAKE-UP PUMP', 'ELECTRIC MOTOR FOR HOT WATER MAKE-UP PUMP',
                    'COOLING WATER MAKE-UP PUMP', 'ELECTRIC MOTOR FOR COOLING WATER MAKE-UP PUMP',
                    'WELL SERVICE PUMP', 'ELECTRIC MOTOR FOR WELL SERVICE PUMP', 
                    'CONTROL PANEL FOR WELL SERVICE PUMP', 'FREQUENCY CONVERTER FOR WELL SERVICE PUMP',
                    'LOW CAPACITY WELL SERVICE PUMP', 'ELECTRIC MOTOR FOR LOW CAPACITY WELL SERVICE PUMP',
                    'CONTROL PANEL FOR LOW CAPACITY WELL SERVICE PUMP',
                    'LEAK TEST PUMP (HOLD)', 'ELECTRIC MOTOR FOR LEAK TEST PUMP (HOLD)'],
    'LOCATION': ['M-15 (Level-3)', 'M-15 (Level-3)', 'M-15 (Level-3)', 'M-15 (Level-3)',
                 'M-10A (Level-1)', 'M-10A (Level-1)', 'M-10A (Level-1)', 'M-17',
                 'M-10A (Level-1)', 'M-10A (Level-1)', 'M-10A (Level-1)',
                 'M-10C(Level-1)', 'M-10C(Level-1)'],
    'AREA_CLASSIFICATION': ['ZONE 2, T3, GROUP llA', 'ZONE 2, T3, GROUP llA', 
                            'NON-HAZARDOUS', 'NON-HAZARDOUS',
                            'ZONE 2 , T3, GROUP llA', 'ZONE 2 , T3, GROUP llA', 'ZONE 2 , T3, GROUP llA', 'NON-HAZARDOUS',
                            'ZONE 2 , T3, GROUP llA', 'ZONE 2 , T3, GROUP llA', 'ZONE 2 , T3, GROUP llA',
                            'ZONE 2 , T3, GROUP llA', 'ZONE 2 , T3, GROUP llA'],
    'PKG_DESCRIPTION': ['RECIPROCATING PUMPS'] * 13
}

df_reciprocating_pumps = pd.DataFrame(data_reciprocating_pumps)
print('Scope of Supply for RECIPROCATING PUMPS')
# display(df_reciprocating_pumps)


# Definição do dataframe do pacote para PRINTED CIRCUIT HEAT EXCHANGER

data_heatric = {
    'TAG': ['P-UC-1231002A/D-01', 'P-UC-1231002A/D-02', 'P-UC-1252001A/B', 'P-UC-1254001A/C-01', 'P-UC-1254001A/C-02'],
    'QTY': ['4 x 33%', '4 x 33%', '2 x 100%', '3 x 50%', '3 x 50%'],
    'DESCRIPTION': ['EXPORTATION GAS COMPRESSION 1ST STAGE DISCHARGE COOLER', 
                   'EXPORTATION GAS COMPRESSION 2ND STAGE DISCHARGE COOLER', 
                   'INJECTION GAS COMPRESSION DISCHARGE COOLER', 
                   'CO2 COMPRESSION - 1ST STAGE DISCHARGE COOLER', 
                   'CO2 COMPRESSION - 2ND STAGE DISCHARGE COOLER'],
    'LOCATION': ['M-07A / M-07B', 'M-07A / M-07B', 'M-07A', 'M-02', 'M-02'],
    'AREA_CLASSIFICATION': ['Zone 2, Group IIA, T3', 'Zone 2, Group IIA, T3', 'Zone 2, Group IIA, T3', 'Zone 2, Group IIA, T3', 'Zone 2, Group IIA, T3'],
    'PKG_DESCRIPTION': ['PRINTED CIRCUIT HEAT EXCHANGER'] * 5
}

df_heatric = pd.DataFrame(data_heatric)

print('Scope of Supply for PRINTED CIRCUIT HEAT EXCHANGER')
# display(df_heatric)

# Definição do dataframe do pacote para GENERAL PURPOSE OFFSHORE CRANES
# Baseado no documento I-RM-3010.2S-5266-631-S2N-001, seção 2.1 SCOPE OF REQUISITION

data_offshore_cranes = {
    'TAG': ['GD-5266501', 'PN-GD-5266501', 'GD-5266502', 'PN-GD-5266502'],
    'QTY': ['1 x100%', '1 x100%', '1 x100%', '1 x100%'],
    'DESCRIPTION': ['AFT FIXED BOOM CRANE', 'CONTROL PANEL', 'FWD FIXED BOOM CRANE', 'CONTROL PANEL'],
    'LOCATION': ['MAIN DECK, M-16', 'M-16', 'MAIN DECK, M-05B', 'M-05B'],
    'AREA_CLASSIFICATION': ['Zone 2, IIA, T3', 'SAFE AREA', 'Zone 2, IIA, T3', 'SAFE AREA'],
    'PKG_DESCRIPTION': ['OFFSHORE CRANE'] * 4
}

df_offshore_cranes = pd.DataFrame(data_offshore_cranes)

print('Scope of Supply for OFFSHORE CRANE')
# display(df_offshore_cranes)

# Lista (string) de dataframes de equipamentos:
list_df_equip = ['df_wip', 'df_lift_pump', 'df_electrochlorination', 'df_N2_generation', 'df_heat_exchangers', 'df_heat_exchangers_prod',
                 'df_progressive_pump', 'df_PIG', 'df_non_api_pump', 'df_api_pump', 'df_fresh_water_maker', 
                 'df_deck_trolley', 'df_flare_system', 'df_fresh_water_chlorination', 'df_reciprocating_pumps', 'df_heatric', 'df_offshore_cranes']

# Converter a lista de nomes em uma lista de DataFrames
list_df_names = [eval(df_name) for df_name in list_df_equip]

# Concatenar todos os dataframes em um único dataframe
df_all_equipment = pd.concat(list_df_names, ignore_index=True)

# Exibir as primeiras linhas do dataframe combinado
# display(df_all_equipment)

# Exibir informações sobre o dataframe combinado
# print(df_all_equipment.info())

Scope of Supply for WATER INJECTION PUMPS AND SEA WATER BOOSTER PUMPS
Scope of Supply for WATER LIFT PUMPS (B-5111001A/D AND B-5111002)
Scope of Supply for SEA WATER ELECTROCHLORINATION UNIT
Scope of Supply for NITROGEN GENERATOR UNITS
Scope of Supply for SHELL AND TUBE HEAT EXCHANGERS (Asvotec)
Scope of Supply for SHELL AND TUBE HEAT EXCHANGERS (Himile)
Scope of Supply for PROGRESSIVE CAVITY PUMPS
Scope of Supply for PIG LAUNCHERS AND RECEIVERS
Scope of Supply for NON-API 610 CENTRIFUGAL PUMPS
Scope of Supply for API 610 CENTRIFUGAL PUMPS
Scope of Supply for FRESH WATER MAKER FOR OIL DILUTION
Scope of Supply for DECK TROLLEY
Scope of Supply for FLARE SYSTEM
Scope of Supply for FRESH WATER CHLORINATION UNIT
Scope of Supply for RECIPROCATING PUMPS
Scope of Supply for PRINTED CIRCUIT HEAT EXCHANGER
Scope of Supply for OFFSHORE CRANE


In [111]:
def count_qty(df):
    count = 0
    for qty_str in df['QTY']:
        parts = qty_str.split()
        # Verificar se a primeira parte é um número válido
        if parts[0].isdigit():
            count += int(parts[0])

    return count

total = 0
data_for_df_equip = []  # Lista para armazenar os dados do novo DataFrame

for df_name in list_df_equip:
   
    df = globals()[df_name]
    qty_count = count_qty(df)
    # Obter o nome do pacote da coluna PKG_DESCRIPTION
    # Assumindo que todos os valores nesta coluna são iguais para um dado DataFrame
    pkg_name = df['PKG_DESCRIPTION'].iloc[0] if not df['PKG_DESCRIPTION'].empty else "Nome do pacote não disponível"

    print(f"Total de equipamentos do pacote {pkg_name}: {qty_count}")
    total += qty_count
    # Adicionar dados para o novo DataFrame 
    data_for_df_equip.append({'PKG_DESCRIPTION': pkg_name, 'QTY_EQUIPMENT': qty_count}) 

print(f"\nTotal de equipamentos: {total}")

# Criar o novo DataFrame df_equip 
df_equip = pd.DataFrame(data_for_df_equip)

Total de equipamentos do pacote WATER INJECTION PUMPS: 22
Total de equipamentos do pacote SEA WATER LIFT PUMP: 21
Total de equipamentos do pacote SEA WATER ELECTROCHLORINATION UNIT: 2
Total de equipamentos do pacote NITROGEN GENERATOR UNITS: 152
Total de equipamentos do pacote SHELL AND TUBE HEAT EXCHANGERS (Asvotec): 27
Total de equipamentos do pacote SHELL AND TUBE HEAT EXCHANGERS (Himile): 6
Total de equipamentos do pacote PROGRESSIVE CAVITY PUMPS: 8
Total de equipamentos do pacote PIG LAUNCHERS AND RECEIVERS: 27
Total de equipamentos do pacote NON-API 610 CENTRIFUGAL PUMPS: 17
Total de equipamentos do pacote API 610 CENTRIFUGAL PUMPS: 90
Total de equipamentos do pacote FRESH WATER MAKER FOR OIL DILUTION: 6
Total de equipamentos do pacote DECK TROLLEY: 10
Total de equipamentos do pacote FLARE SYSTEM: 8
Total de equipamentos do pacote FRESH WATER CHLORINATION UNIT: 2
Total de equipamentos do pacote RECIPROCATING PUMPS: 28
Total de equipamentos do pacote PRINTED CIRCUIT HEAT EXCHANGER

### Importando arquivos

In [112]:
ld_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_consolidada_have_a_comment.xlsx'
mapa_topsideP84_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2025-04-21 P84 Procurement Map Topside.xlsx'
mapa_topsideP85_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2025-04-21 P85 Procurement Map Topside.xlsx'
mapa_orbis_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\orbis.xlsx'
comunication_matrix_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2024-03-10 - P84-P85 Package Communication Matrix.xlsx'
integra_eng_docs = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\RE-General Query - Technical Engineering Documents.xlsx'
vda_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\VDA\\vda.xlsx'
avanco_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\Avanço Pacotes.xlsx'
notes_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\notes.xlsx'

df_LD = pd.read_excel(ld_file, sheet_name="LD")
df_mapa_topsideP84 = pd.read_excel(mapa_topsideP84_file, sheet_name="P84 Topside Map")
df_mapa_topsideP85 = pd.read_excel(mapa_topsideP85_file, sheet_name="P85 Topside Map")
df_mapa_orbis = pd.read_excel(mapa_orbis_file, sheet_name="Export")
df_mc = pd.read_excel(comunication_matrix_file, sheet_name="Package Matrix")
df_integra = pd.read_excel(integra_eng_docs, sheet_name="DADOS")
# cria um dataframe chamado df_integra_last somente com os documentos na última versão:
df_integra_last = df_integra.loc[df_integra.groupby('CODE')['VERSION'].idxmax()]
df_VDA = pd.read_excel(vda_file, sheet_name="Sheet1")
df_avanco = pd.read_excel(avanco_file, sheet_name="PREENCHIMENTO AVANÇO")
df_notes = pd.read_excel(notes_file, sheet_name="notes")


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


#### Pré-Processamento dos dados

In [113]:
# Preparação dos DataFrames para limpeza de espaços em branco

df_mapa_orbis.rename(columns = {"PKG DESCRIPTION":"PKG_DESCRIPTION", "Pkg ID":"Pkg_ID", "PO Issue Plan":"PO_Issue_Date_Plan", "Last deliv. Plan/Forecast":"Required_On_Site_Date_Plan"}, inplace = True)
df_mc.rename(columns={"Package Engineer_Petrobras":"Package_Engineer_Petrobras"}, inplace = True)
df_mc.rename(columns={"Package Engineer_Engineering":"Package_Engineer_Engineering"}, inplace = True)
df_mc.rename(columns={"EPC_Responsible_(Expeditor)":"Package_Expeditor"}, inplace = True)
df_mc.rename(columns={"Package Engineer_KBR":"Package_KBR"}, inplace = True)
df_avanco.rename(columns={"PKG DESCRIPTION":"PKG_DESCRIPTION", "%PLAN":"PLAN", "%REAL":"REAL", "AVANÇO":"DESVIO"}, inplace = True)
df_avanco = df_avanco.drop(columns=['Unnamed: 6'])

# limpeza de espaços em branco para todas as colunas de cada DataFrame da lista abaixo:
# Lista de DataFrames
dataframes = [df_LD, df_mapa_topsideP84, df_mapa_topsideP85, df_mapa_orbis, df_mc]

# Função para limpar espaços em branco
def limpar_espacos(df):
    return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

def limpar_espacos__entre_palavras(df):
    return df.applymap(lambda x: re.sub(' +', ' ', str(x)) if isinstance(x, str) else x)

# Aplicar a limpeza para cada DataFrame na lista
for i, df in enumerate(dataframes):
    dataframes[i] = limpar_espacos(df)

# Atribuir os DataFrames limpos de volta às variáveis originais 
df_LD, df_mapa_topsideP84, df_mapa_topsideP85, df_mapa_orbis, df_mc = dataframes
 
print("Limpeza de espaços em branco concluída para todos os DataFrames.")

# Aplicar a limpeza para cada DataFrame na lista
for i, df in enumerate(dataframes):
    dataframes[i] = limpar_espacos__entre_palavras(df)
 
# Atribuir os DataFrames limpos de volta às variáveis originais 
df_LD, df_mapa_topsideP84, df_mapa_topsideP85, df_mapa_orbis, df_mc = dataframes

print("Limpeza de espaços em branco concluída para todos os DataFrames (entre palavras).")

print("Ajustando o dataframe df_avanco para remover - HULL e - TOPSIDE.")
df_avanco['PKG_DESCRIPTION'] = df_avanco['PKG_DESCRIPTION'].str.replace(' - HULL', '').str.replace(' - TOPSIDE', '')


Limpeza de espaços em branco concluída para todos os DataFrames.
Limpeza de espaços em branco concluída para todos os DataFrames (entre palavras).
Ajustando o dataframe df_avanco para remover - HULL e - TOPSIDE.


### MAPA DE SUPRIMENTOS DO TOPSIDE:
- Percorre os 2 dataframes e atualiza o df_escopo

In [114]:
lista_dfs = ["df_mapa_topsideP84", "df_mapa_topsideP85"]

print("Atualizando informações dos Mapas de Suprimentos...")
for item in lista_dfs:
    df = eval(item)  # Converte o nome da variável em um objeto DataFrame
    print(f"Processing DataFrame: {item}")
    
    for row in df.itertuples():
        pkg_description = row.PKG_DESCRIPTION
        po_number = row.PO_Number
        # usando a função .loc para atualizar os valores
        if po_number:
            df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "PO_Number"] = po_number

print("Atualizando informações do Orbis...")
for row in df_mapa_orbis.itertuples():
    pkg_description = row.PKG_DESCRIPTION
    pkg_ID = row.Pkg_ID
    PO_Issue_Date_Plan = row.PO_Issue_Date_Plan
    Required_On_Site_Date_Plan = row.Required_On_Site_Date_Plan
    # if pkg_ID:
    df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "PO_Issue_Date_Plan"] = PO_Issue_Date_Plan
    df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "Required_On_Site_Date_Plan"] = Required_On_Site_Date_Plan

print("Atualizando informações do Communication Matrix...")
for row in df_mc.itertuples():
    pkg_description = row.PKG_DESCRIPTION
    Package_KBR = row.Package_KBR
    Package_Seatrium = row.Package_Engineer_Engineering
    Package_Expeditor = row.Package_Expeditor
    # if pkg_ID:
    df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "Package_KBR"] = Package_KBR
    df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "Package_Seatrium"] = Package_Seatrium
    df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "Package_Expeditor"] = Package_Expeditor

print("Atualiza informações do Integra...")

for row in df_integra_last.itertuples():
    document_emitido = row.CODE
    
    # Processa MR_Number que podem ter múltiplos valores
    for index, mr_value in df_db['MR_Number'].items():
        if pd.notna(mr_value):  # Verifica se o valor não é NaN
            # Divide os valores separados por vírgula e remove espaços
            mr_list = [item.strip() for item in str(mr_value).split(',')]
            # Verifica se o documento emitido está na lista
            if document_emitido in mr_list:
                df_db.loc[index, "MR_Emitida"] = "Sim"
    
    # Processa TBE_Number que podem ter múltiplos valores
    for index, tbe_value in df_db['TBE_Number'].items():
        if pd.notna(tbe_value):  # Verifica se o valor não é NaN
            # Divide os valores separados por vírgula e remove espaços
            tbe_list = [item.strip() for item in str(tbe_value).split(',')]
            # Verifica se o documento emitido está na lista
            if document_emitido in tbe_list:
                df_db.loc[index, "TBE_Emitida"] = "Sim"

print("Atualiza informações das quantidades de equipamentos...")
# Update df_db with equipment quantities from df_equip
for index, row in df_equip.iterrows():
    pkg_description = row['PKG_DESCRIPTION']
    qty_equipment = row['QTY_EQUIPMENT']
    
    # Update the corresponding rows in df_db
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'QTY_EQUIPMENT'] = qty_equipment

print("Atualiza informações do Avanço Geral...")
# Substituir NaN por 0 nas colunas abaixo
df_avanco['PLAN'] = df_avanco['PLAN'].fillna(0)
df_avanco['REAL'] = df_avanco['REAL'].fillna(0)
df_avanco['DESVIO'] = df_avanco['DESVIO'].fillna(0)

# Filtra apenas os registros do FPSO P84
df_avanco_p84 = df_avanco[df_avanco['FPSO'] == 'P84']
# Ordena por data em ordem crescente para garantir que o último valor seja o mais recente
df_avanco_p84 = df_avanco_p84.sort_values(by='DATE')
# Obtém os últimos valores para cada PKG_DESCRIPTION
df_latest_p84 = df_avanco_p84.drop_duplicates(subset=['PKG_DESCRIPTION'], keep='last')
# Update df_db com os valores mais recentes
for index, row in df_latest_p84.iterrows():
    pkg_description = row['PKG_DESCRIPTION']
    plan = row['PLAN']
    real = row['REAL']
    desvio = row['DESVIO']
    # Update the corresponding rows in df_db
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'PLAN_P84'] = plan
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'REAL_P84'] = real
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'DESVIO_P84'] = desvio

# Filtra apenas os registros do FPSO P84
df_avanco_p85 = df_avanco[df_avanco['FPSO'] == 'P85']
# Ordena por data em ordem crescente para garantir que o último valor seja o mais recente
df_avanco_p85 = df_avanco_p85.sort_values(by='DATE')
# Obtém os últimos valores para cada PKG_DESCRIPTION
df_latest_p85 = df_avanco_p85.drop_duplicates(subset=['PKG_DESCRIPTION'], keep='last')
# Update df_db com os valores mais recentes
for index, row in df_latest_p85.iterrows():
    pkg_description = row['PKG_DESCRIPTION']
    plan = row['PLAN']
    real = row['REAL']
    desvio = row['DESVIO']
    # Update the corresponding rows in df_db
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'PLAN_P85'] = plan
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'REAL_P85'] = real
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'DESVIO_P85'] = desvio

# Substituir NaN por 0:
df_db['PLAN_P85'] = df_db['PLAN_P85'].fillna(0)
df_db['REAL_P85'] = df_db['REAL_P85'].fillna(0)
df_db['DESVIO_P85'] = df_db['DESVIO_P85'].fillna(0)

df_db['PLAN_P84'] = df_db['PLAN_P84'].fillna(0)
df_db['REAL_P84'] = df_db['REAL_P84'].fillna(0)
df_db['DESVIO_P84'] = df_db['DESVIO_P84'].fillna(0)


Atualizando informações dos Mapas de Suprimentos...
Processing DataFrame: df_mapa_topsideP84
Processing DataFrame: df_mapa_topsideP85
Atualizando informações do Orbis...
Atualizando informações do Communication Matrix...
Atualiza informações do Integra...
Atualiza informações das quantidades de equipamentos...
Atualiza informações do Avanço Geral...


### Documentos em atraso com relação a última revisão da LD

In [115]:
from datetime import datetime

# Get today's date
today = datetime.now().date()

# Create a list to store delayed documents
delayed_docs = []

# Iterate through VDA documents
for index, row in df_LD.iterrows():
    client_document = row['CLIENT_DOCUMENT']
    planned_date = row['PLANNED DATE'].date()
    
    # Check if document exists in df_integra_last
    doc_issued = client_document in df_integra_last['CODE'].values
    
    # Document is delayed if not issued and planned date has passed
    if not doc_issued and planned_date < today:
        delayed_docs.append({
            'CLIENT_DOCUMENT': client_document,
            'DOCUMENT_TITLE': row['DOCUMENT_TITLE'],
            'PLANNED DATE': planned_date,
            'DAYS_DELAYED': (today - planned_date).days
        })

# Create DataFrame with delayed documents
df_atrasados = pd.DataFrame(delayed_docs)

# Sort by days delayed (descending)
if not df_atrasados.empty:
    df_atrasados = df_atrasados.sort_values('DAYS_DELAYED', ascending=False)

## Avanço dos Pacotes

#### Filtra todos documentos e documentos emitidos dos pacotes e realiza uma estratificação

In [ ]:
import pandas as pd
import re

def criar_regex_pattern(*textos_busca):
    return ''.join(f"(?=.*{re.escape(texto)})" for texto in textos_busca if texto)

def processar_sistemas(row, df_LD, df_integra_last, df_atrasados):
    pkg_description = row.name
    trigrama = row['Trigram']
    
    # Handle multiple systems separated by commas
    sistemas = row['System'].split(',') if pd.notna(row['System']) else []
    sistemas = [s.strip() for s in sistemas]
    
    total_docs = 0
    total_docs_emitidos = 0
    
    # Process each system
    for sistema in sistemas:
        if pd.notna(trigrama) and pd.notna(sistema):
            regex_pattern = criar_regex_pattern(sistema, trigrama)
            
            # Filter documents
            df_Package = df_LD[df_LD['CLIENT_DOCUMENT'].str.contains(regex_pattern, regex=True, na=False)]
            df_Emitido = df_integra_last[df_integra_last['CODE'].str.contains(regex_pattern, regex=True, na=False)]
            df_DocsAtrasados = df_atrasados[df_atrasados['CLIENT_DOCUMENT'].str.contains(regex_pattern, regex=True, na=False)]
            
            total_docs += len(df_Package)
            total_docs_emitidos += len(df_Emitido)
            total_docs_em_atraso = len(df_DocsAtrasados)
    
    return pd.Series({
        'N_DOCS': total_docs,
        'N_DOCS_EMITIDOS': total_docs_emitidos,
        'N_DOCS_EM_ATRASO': total_docs_em_atraso
    })

# Process all packages
df_resultados = df_db.apply(lambda row: processar_sistemas(row, df_LD, df_integra_last, df_atrasados), axis=1)

# Update df_db with results
df_db[['N_DOCS', 'N_DOCS_EMITIDOS', 'N_DOCS_EM_ATRASO']] = df_resultados


### Comentário em Atraso: Seatrium vs Petrobras

In [117]:
from datetime import datetime, timedelta
import numpy as np
from pandas.tseries.offsets import BusinessDay

# Get today's date
today = datetime.now().date()

# Set different tolerance days for each company
petrobras_tolerance_days = 8
seatrium_tolerance_days = 7

# Create lists to store delayed documents for each company
seatrium_delays = []
petrobras_delays = []

# Function to calculate business days between two dates
def business_days_between(start_date, end_date):
    # Convert to datetime if they are not
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, '%Y-%m-%d').date()
    
    # Calculate business days
    business_days = np.busday_count(
        start_date.strftime('%Y-%m-%d'),
        end_date.strftime('%Y-%m-%d')
    )
    return business_days

# Find the latest version for each document 'Code' in df_VDA
df_VDA_latest = df_VDA.loc[df_VDA.groupby('Code')['Version'].idxmax()]

# Check Petrobras delays (documents in VDA with status "EM ANALISE" for more than 8 business days)
for _, row in df_VDA_latest.iterrows():
    if row['Workflow Status'] == 'EM ANALISE':
        creation_date = row['Creation Date'].date()
        business_days = business_days_between(creation_date, today)
        if business_days > petrobras_tolerance_days:
            petrobras_delays.append({
                'document': row['Reference Document'],
                'vda': row['Code'],
                'title': row['Title'],
                'version': row['Version'],
                'workflow_status': row['Workflow Status'],
                'creation_date': row['Creation Date'],
                'delay': business_days
            })

# Check Seatrium delays (documents in LD but not in VDA and created more than 7 business days ago)
for _, row in df_LD.iterrows():
    client_document = row['CLIENT_DOCUMENT']
    # Check if document exists in df_integra_last
    doc_issued = client_document in df_integra_last['CODE'].values
    if doc_issued:
        # Check if document is in VDA
        doc_in_vda = client_document in df_VDA['Reference Document'].values
        if not doc_in_vda:
            # Get creation date from df_integra_last
            creation_date = df_integra_last[df_integra_last['CODE'] == client_document]['CREATION DATE'].iloc[0].date()
            business_days = business_days_between(creation_date, today)
            if business_days > seatrium_tolerance_days:
                seatrium_delays.append({
                    'document': client_document,
                    'title': row['TITLE'],
                    'planned_date': row['PLANNED DATE'],
                    'discipline': row['Discipline'],
                    'comment': row['Have_comment'],
                    'creation_date': creation_date,
                    'delay': business_days
                })

# Create DataFrames for delayed documents
df_seatrium_delays = pd.DataFrame(seatrium_delays)
df_petrobras_delays = pd.DataFrame(petrobras_delays)

# Function to count delays for each package
def count_package_delays(pkg_row, df_delays):
    count = 0
    trigrama = pkg_row['Trigram']
    sistemas = pkg_row['System'].split(',') if pd.notna(pkg_row['System']) else []
    sistemas = [s.strip() for s in sistemas]
    for sistema in sistemas:
        if pd.notna(trigrama) and pd.notna(sistema):
            # Create regex pattern
            pattern = f".*{sistema}.*{trigrama}.*"
            # Count matching documents
            matching_docs = df_delays[df_delays['document'].str.contains(pattern, regex=True, na=False)]
            count += len(matching_docs)
    return count

# Mapeia coluna Discipline no df_petrobras_delays
# Create a mapping dictionary from df_LD
discipline_map = df_LD.set_index('CLIENT_DOCUMENT')['Discipline'].to_dict()
# Add new Disciplina column to df_petrobras_delays
df_petrobras_delays['Discipline'] = df_petrobras_delays['document'].map(discipline_map)
# Sort by Disciplina and delay
df_petrobras_delays = df_petrobras_delays.sort_values(['Discipline', 'delay'], ascending=[True, False])

# Add new columns to df_db
df_db['Atraso_KBR'] = df_db.apply(lambda row: count_package_delays(row, df_seatrium_delays), axis=1)
df_db['Atraso_PB'] = df_db.apply(lambda row: count_package_delays(row, df_petrobras_delays), axis=1)

# Definimos a nova ordem das colunas em uma lista
new_order = ['PKG_DESCRIPTION',	'QTY_EQUIPMENT', 'Discipline', 'Critical', 'System', 'Trigram', 'Vendor', 
             'MR_Number', 'MR_Emitida', 'TBE_Number', 'TBE_Emitida', 'PO_Number', 'KOM_DATE', 'PIM_DATE',	
             'DR_MOTORS_DATE', 'PLAN_DELIVERY_DATE', 'PO_Issue_Date_Plan', 'Required_On_Site_Date_Plan',	
             'Package_KBR',	'Package_Seatrium',	'Package_Expeditor', 'N_DOCS', 'N_DOCS_EMITIDOS', 
             'N_DOCS_EM_ATRASO', 'Atraso_KBR', 'Atraso_PB', 'PLAN_P84', 'REAL_P84', 'DESVIO_P84', 'PLAN_P85', 'REAL_P85', 'DESVIO_P85']

# Usamos essa lista para reordenar o DataFrame
df_db = df_db[new_order]

# Display updated df_db
display(df_db[['PKG_DESCRIPTION', 'N_DOCS', 'N_DOCS_EMITIDOS', 'N_DOCS_EM_ATRASO', 'Atraso_KBR', 'Atraso_PB', 'PLAN_P84', 'REAL_P84', 'DESVIO_P84', 'PLAN_P85', 'REAL_P85', 'DESVIO_P85']])


,PKG_DESCRIPTION,N_DOCS,N_DOCS_EMITIDOS,N_DOCS_EM_ATRASO,Atraso_KBR,Atraso_PB,PLAN_P84,REAL_P84,DESVIO_P84,PLAN_P85,REAL_P85,DESVIO_P85
0,API 610 CENTRIFUGAL PUMPS,646,52,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
1,DECK TROLLEY,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
2,FLARE SYSTEM,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
3,FRESH WATER CHLORINATION UNIT,190,55,26,1,3,22.05,10.76,-11.29,11.59,6.59,-5.00
4,NITROGEN GENERATOR UNITS,179,88,28,13,2,21.10,19.90,-1.20,0.00,0.00,0.00
5,NON-API 610 CENTRIFUGAL PUMPS,384,30,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
6,PIG LAUNCHERS AND RECEIVERS,125,1,0,1,0,0.00,0.00,0.00,0.00,0.00,0.00
7,PROGRESSIVE CAVITY PUMPS,90,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
8,RECIPROCATING PUMPS,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
9,SEA WATER ELECTROCHLORINATION UNIT,237,68,6,4,0,22.05,10.76,-11.29,11.59,6.59,-5.00


### Plota Gráfico dos Comentários em Atraso

In [118]:
import numpy as np
import matplotlib.pyplot as plt

# Defina o tamanho da figura
plt.figure(figsize=(16, 6))

# Largura das barras
bar_width = 0.2

# Posições das barras no eixo x
r1 = np.arange(len(df_db['PKG_DESCRIPTION']))
r2 = [x + bar_width for x in r1]

# Plote o gráfico de barras
plt.bar(r1, df_db['Atraso_KBR'], width=bar_width, label='Overdue - KBR', color='b')
plt.bar(r2, df_db['Atraso_PB'], width=bar_width, label='Overdue - Petrobras', color='g')

# Adicione rótulos e título
plt.xlabel('Package Description')
plt.ylabel('Overdue Documents')
plt.title('Overdue Comments by Package')

# Rotacione os rótulos do eixo x para melhor legibilidade
plt.xticks([r + bar_width for r in range(len(df_db['PKG_DESCRIPTION']))], df_db['PKG_DESCRIPTION'], rotation=45, ha='right')

# Adiciona a legenda
plt.legend()

# Adiciona os valores acima das barras
for i in range(len(df_db['PKG_DESCRIPTION'])):
    if df_db['Atraso_KBR'][i] != 0:
        plt.text(r1[i], df_db['Atraso_KBR'][i], str(df_db['Atraso_KBR'][i]), ha='center', va='bottom')
    if df_db['Atraso_PB'][i] != 0:
        plt.text(r2[i], df_db['Atraso_PB'][i], str(df_db['Atraso_PB'][i]), ha='center', va='bottom')

# Ajuste o layout para evitar cortes
plt.tight_layout()

# Mostre o gráfico
# plt.show()
figures = []
# Captura o gráfico
figures.append(plt.gcf())  # Captura a figura atual
plt.close()

### Plota Gráfico dos Documentos em Atraso

In [119]:
import numpy as np
import matplotlib.pyplot as plt

# Defina o tamanho da figura
plt.figure(figsize=(16, 6))

# Largura das barras
bar_width = 0.2

# Posições das barras no eixo x
r1 = np.arange(len(df_db['PKG_DESCRIPTION']))
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]

# Plote o gráfico de barras
plt.bar(r1, df_db['N_DOCS'], width=bar_width, label='Total Number of Documents', color='b')
plt.bar(r2, df_db['N_DOCS_EMITIDOS'], width=bar_width, label='Issued Documents', color='g')
plt.bar(r3, df_db['N_DOCS_EM_ATRASO'], width=bar_width, label='Delayed Documents', color='r')

# Adicione rótulos e título
plt.xlabel('Package Description')
plt.ylabel('Number of Documents')
plt.title('Number of Documents by Package')

# Rotacione os rótulos do eixo x para melhor legibilidade
plt.xticks([r + bar_width for r in range(len(df_db['PKG_DESCRIPTION']))], df_db['PKG_DESCRIPTION'], rotation=45, ha='right')

# Adiciona a legenda
plt.legend()

# Adiciona os valores acima das barras
for i in range(len(df_db['PKG_DESCRIPTION'])):
    # Mostra o valor apenas se for diferente de zero
    if df_db['N_DOCS'][i] != 0:
        plt.text(r1[i], df_db['N_DOCS'][i], str(df_db['N_DOCS'][i]), ha='center', va='bottom')
    if df_db['N_DOCS_EMITIDOS'][i] != 0:
        plt.text(r2[i], df_db['N_DOCS_EMITIDOS'][i], str(df_db['N_DOCS_EMITIDOS'][i]), ha='center', va='bottom')
    if df_db['N_DOCS_EM_ATRASO'][i] != 0:
        plt.text(r3[i], df_db['N_DOCS_EM_ATRASO'][i], str(df_db['N_DOCS_EM_ATRASO'][i]), ha='center', va='bottom')

# Ajuste o layout para evitar cortes
plt.tight_layout()

# Captura o gráfico
figures.append(plt.gcf())  # Captura a figura atual
plt.close()


### Plota Gráfico do Avanço Geral por Pacote - P84

In [120]:
import numpy as np
import matplotlib.pyplot as plt

# Defina o tamanho da figura
plt.figure(figsize=(16, 6))

# Largura das barras
bar_width = 0.2

# Posições das barras no eixo x
r1 = np.arange(len(df_db['PKG_DESCRIPTION']))
r2 = [x + bar_width for x in r1]

# Plote o gráfico de barras
plt.bar(r1, df_db['PLAN_P84'], width=bar_width, label='PLAN_P84', color='b')
plt.bar(r2, df_db['REAL_P84'], width=bar_width, label='REAL_P84', color='g')

# Adicione rótulos e título
plt.xlabel('Package Description')
plt.ylabel('Progress (%)')
plt.title('Overall Progress by Package - P84')

# Rotacione os rótulos do eixo x para melhor legibilidade
plt.xticks([r + bar_width for r in range(len(df_db['PKG_DESCRIPTION']))], df_db['PKG_DESCRIPTION'], rotation=45, ha='right')

# Adiciona a legenda
plt.legend()

# Adiciona os valores acima das barras
for i in range(len(df_db['PKG_DESCRIPTION'])):
    # Mostra o valor apenas se for diferente de zero
    if df_db['PLAN_P84'][i] != 0:
        plt.text(r1[i], df_db['PLAN_P84'][i], str(df_db['PLAN_P84'][i]), ha='center', va='bottom')
    if df_db['REAL_P84'][i] != 0:
        plt.text(r2[i], df_db['REAL_P84'][i], str(df_db['REAL_P84'][i]), ha='center', va='bottom')

# Ajuste o layout para evitar cortes
plt.tight_layout()
# Captura o gráfico
figures.append(plt.gcf())  # Captura a figura atual
plt.close()

### Plota Gráfico do Avanço Geral por Pacote - P85

In [121]:
import numpy as np
import matplotlib.pyplot as plt

# Defina o tamanho da figura
plt.figure(figsize=(16, 6))

# Largura das barras
bar_width = 0.2

# Posições das barras no eixo x
r1 = np.arange(len(df_db['PKG_DESCRIPTION']))
r2 = [x + bar_width for x in r1]

# Plote o gráfico de barras
plt.bar(r1, df_db['PLAN_P85'], width=bar_width, label='PLAN_P85', color='b')
plt.bar(r2, df_db['REAL_P85'], width=bar_width, label='REAL_P85', color='g')

# Adicione rótulos e título
plt.xlabel('Package Description')
plt.ylabel('Progress (%)')
plt.title('Overall Progress by Package - P85')

# Rotacione os rótulos do eixo x para melhor legibilidade
plt.xticks([r + bar_width for r in range(len(df_db['PKG_DESCRIPTION']))], df_db['PKG_DESCRIPTION'], rotation=45, ha='right')

# Adiciona a legenda
plt.legend()

# Adiciona os valores acima das barras
for i in range(len(df_db['PKG_DESCRIPTION'])):
    # Mostra o valor apenas se for diferente de zero
    if df_db['PLAN_P85'][i] != 0:
        plt.text(r1[i], df_db['PLAN_P85'][i], str(df_db['PLAN_P85'][i]), ha='center', va='bottom')
    if df_db['REAL_P85'][i] != 0:
        plt.text(r2[i], df_db['REAL_P85'][i], str(df_db['REAL_P85'][i]), ha='center', va='bottom')

# Ajuste o layout para evitar cortes
plt.tight_layout()

# Mostre o gráfico
# plt.show()
# Captura o gráfico
figures.append(plt.gcf())  # Captura a figura atual
plt.close()


#### Salva os dados em Arquivo Excel

In [122]:
control_cesar = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2025-04-25_P84-P85_controle_cesar.xlsx'

# Salvando arquivos P84 e P85:
with pd.ExcelWriter(control_cesar) as writer:
    df_integra.to_excel(writer, sheet_name='integra_all_eng_docs', index=False)
    df_db.to_excel(writer, sheet_name='Resumo', index=False)
    df_all_equipment.to_excel(writer, sheet_name='equip', index=False)
    df_atrasados.to_excel(writer, sheet_name='docs_atrasados', index=False)
    df_petrobras_delays.to_excel(writer, sheet_name='petrobras_delays', index=False)
    df_seatrium_delays.to_excel(writer, sheet_name='seatrium_delays', index=False)
print(f'Arquivo {control_cesar} salvo em: ', Path.cwd())


Arquivo C:\Users\elxy\Documents\Codigos\Python\P84_85\2025-04-25_P84-P85_controle_cesar.xlsx salvo em:  c:\Users\ELXY\Documents\Codigos\Python\P84_85


### Gera relatório de avanço

In [123]:
import pandas as pd
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle
from reportlab.lib.units import inch, cm
import datetime

def generate_report_from_existing(df_notes, figures, output_filename):
    """
    Gera um relatório PDF baseado em um dataframe de notas e figuras.
    
    Args:
        df_notes (pandas.DataFrame): DataFrame contendo as notas para o relatório com colunas:
                                    'PKG_DESCRIPTION', 'DATE', 'Fatos_Relevantes', 'Pontos_Atencao'
        figures (list): Lista de objetos de figura (matplotlib ou caminho para imagens)
        output_filename (str): Nome do arquivo PDF de saída
    """
    # Configurar o documento em tamanho A4 com margens estreitas
    page_width, page_height = A4
    margin = 1.0 * cm  # 1 cm de margem em todos os lados
    
    doc = SimpleDocTemplate(
        output_filename,
        pagesize=A4,
        rightMargin=margin,
        leftMargin=margin,
        topMargin=margin,
        bottomMargin=margin
    )
    
    styles = getSampleStyleSheet()
    
    # Calcular largura disponível para conteúdo
    content_width = page_width - (2 * margin)
    
    # Criar estilos personalizados com nomes únicos
    title_style = ParagraphStyle(
        name='CustomTitle',
        parent=styles['Heading1'],
        fontSize=16,
        alignment=1,  # centralizado
        spaceAfter=12
    )
    
    subtitle_style = ParagraphStyle(
        name='CustomSubtitle',
        parent=styles['Heading2'],
        fontSize=12,
        alignment=1,  # centralizado
        spaceAfter=12
    )
    
    section_title_style = ParagraphStyle(
        name='CustomSectionTitle',
        parent=styles['Heading2'],
        fontSize=14,
        spaceAfter=6
    )
    
    package_title_style = ParagraphStyle(
        name='CustomPackageTitle',
        parent=styles['Heading3'],
        fontSize=11,
        fontName='Helvetica-Bold',
        spaceAfter=4,
        spaceBefore=10
    )
    
    bullet_point_style = ParagraphStyle(
        name='CustomBulletPoint',
        parent=styles['Normal'],
        fontSize=10,
        leftIndent=20,
        bulletIndent=10,
        spaceAfter=2
    )
    
    sub_header_style = ParagraphStyle(
        name='CustomSubHeader',
        parent=styles['Normal'],
        fontSize=11,
        fontName='Helvetica-Bold',
        spaceAfter=2,
        spaceBefore=6
    )
    
    # Conteúdo do documento
    elements = []
    
    # Título
    elements.append(Paragraph("Progress Report - CÉSAR SILVESTRE", title_style))
    
    # Data
    current_date = datetime.datetime.now().strftime("%d/%m/%Y")
    elements.append(Paragraph(f"Date: {current_date}", subtitle_style))
    elements.append(Spacer(1, 0.25*inch))
    
    # Seção 1: Indicadores
    elements.append(Paragraph("1. Metrics:", section_title_style))
    elements.append(Spacer(1, 0.1*inch))
    
    # Adicionar as figuras uma embaixo da outra
    if len(figures) > 0:
        # Salvar temporariamente as figuras matplotlib se necessário
        image_paths = []
        for i, fig in enumerate(figures):
            if hasattr(fig, 'savefig'):  # Se for uma figura matplotlib
                temp_path = f'temp_figure_{i}.png'
                fig.savefig(temp_path, dpi=100, bbox_inches='tight')
                image_paths.append(temp_path)
            else:  # Se for um caminho para uma imagem
                image_paths.append(fig)
        
        # Definir dimensões para aproveitar melhor o espaço A4
        fig_width = content_width  # Usa toda a largura disponível
        fig_height = content_width * 0.55  # Proporção aproximada para gráficos
        
        # Adicionar cada figura individualmente
        for img_path in image_paths:
            img = Image(img_path, width=fig_width, height=fig_height)
            elements.append(img)
            elements.append(Spacer(1, 0.2*inch))  # Espaço entre figuras
    
    elements.append(Spacer(1, 0.3*inch))
    
    # Seção 2: Comentários
    elements.append(Paragraph("2. Comments:", section_title_style))
    elements.append(Spacer(1, 0.1*inch))
    
    # Processar cada pacote do dataframe
    for _, row in df_notes.iterrows():
        pkg_description = row['PKG_DESCRIPTION']
        elements.append(Paragraph(pkg_description, package_title_style))
        
        # Fatos relevantes
        if pd.notna(row['Fatos_Relevantes']):
            elements.append(Paragraph("<b><i>Relevant Facts:</i></b>", sub_header_style))
            
            # Processar os itens com marcadores
            facts = row['Fatos_Relevantes'].split(';')
            for fact in facts:
                fact = fact.strip()
                if fact:
                    elements.append(Paragraph(f"• {fact}", bullet_point_style))
        
        # Pontos de atenção
        if pd.notna(row['Pontos_Atencao']):
            elements.append(Paragraph("<b><i>Concerns:</i></b>", sub_header_style))
            
            # Processar os itens com marcadores
            points = row['Pontos_Atencao'].split(';')
            for point in points:
                point = point.strip()
                if point:
                    elements.append(Paragraph(f"• {point}", bullet_point_style))
        elements.append(Spacer(1, 0.1*inch))
    
    # Gerar o PDF
    doc.build(elements)
    
    print(f"Relatório gerado com sucesso: {output_filename}")


### Gera Relatórios

In [124]:
print("Gerando relatório para o Robson...")

# Gere o relatório
generate_report_from_existing(df_notes, figures, '2025-04-25 Relatorio de Avanço - Cesar.pdf')


Gerando relatório para o Robson...
Relatório gerado com sucesso: 2025-04-25 Relatorio de Avanço - Cesar.pdf
